# This is where the magic happens

- We assign characters to laughter instances (decide who was responsible for the laughter)
- We create a custom subtitle file for each episode to visually inspect (demo) the output
- We create charts and other visualizations for the writeup

In [ ]:
import sqlite3
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import chart_studio.tools as tls
import pandas as pd

### Choosing colors to represent each main character for the graphs later on

In [ ]:
# graph colors
chandlercolor = 'indianred'
joeycolor = 'dodgerblue'
monicacolor = 'gold'
phoebecolor = 'orchid'
rachelcolor = 'lightskyblue'
rosscolor = 'limegreen'

### Plotly sign-in

In [ ]:
# plotly sign-in
username = 'redacted'
api_key = 'redacted'

### Building dict with seasons and episodes in each season to loop through

In [ ]:
seasepdict = {}

conn = sqlite3.connect('/Users/Jack/Developer/friends/friendsdb.sqlite')
cur = conn.cursor()

cur.execute('''SELECT DISTINCT season FROM laughs''')
for value in cur:
    seasepdict[value[0]] = 0

for season, episodes in seasepdict.items():
    cur.execute('''SELECT MAX(episode) FROM laughs WHERE season=?''', (season,))
    for value in cur:
        seasepdict[season] = value[0]
print(seasepdict)


### Utility functions for processing SQLite data

In [ ]:
def timeconvert(string):
    hours = int(string[0:2])
    mins = int(string[3:5])
    secs = int(string[6:8])
    ms = int(string[9:])
    intoms = ms + (secs * 1000) + (mins * 60 * 1000) + (hours * 60 * 60 * 1000)
    return intoms

def linesplit(linenum, begtime, endtime):
    if not isinstance(linenum, int) and "A" in linenum:
        endtime = begtime + ((endtime - begtime)/2)
    elif not isinstance(linenum, int) and "B" in linenum:
        begtime = endtime - ((endtime - begtime)/2)
    return begtime, endtime

### Laughter Attribution

- We process the SQLite data then we use a minimization function to decide which character is responsible for each laugh
- The minimization function takes the ending timestamp of each subtitle line and the beginning timestamp of each laugh and finds the subtitle line with the smallest distance to the beginning of a laugh, then assigns that character to the laugh

In [ ]:
speakdict = {}
laughdict = {}

for season, episodes in seasepdict.items():
    for epnum in range(0, episodes):
        epnum += 1
#         if season == 1 and epnum == 1: # remove later
        speaklist = []
        laughlist = []
        cur.execute('''SELECT starttime, endtime, x, y, linenum FROM subs WHERE season=? AND episode=?''', (season, epnum))
        for info in cur:
            begtime = timeconvert(info[0])
            endtime = timeconvert(info[1])
            x = info[2].lower()
            y = info[3].lower()
            linenum = info[4]
            begtime, endtime = linesplit(linenum, begtime, endtime)
            if y != '':
                templist = [begtime, endtime, y, linenum]
            else:
                templist = [begtime, endtime, x, linenum]
            speaklist.append(templist)
#             print(speaklist)
        cur.execute('''SELECT beg, end FROM laughs WHERE season=? AND episode=?''', (season, epnum))
        for laughtimes in cur:
            templist = [laughtimes[0], laughtimes[1], None]
            laughlist.append(templist)
#             print(laughlist)
        # need to fix lines with A and B
        for laugh in laughlist:
            closest = 5000
            for info in speaklist:
                laughstart = laugh[0]
                subend = info[1]
                char = info[2]
                proximity = abs(laughstart - subend)
                if proximity < closest:
                    closest = proximity
                    laugh[2] = char
        speakdict[str(season).zfill(2) + str(epnum).zfill(2)] = speaklist
        laughdict[str(season).zfill(2) + str(epnum).zfill(2)] = laughlist             


### Inputting laughter attributions into SQLite

In [ ]:
for seasep, list in laughdict.items():
    season = int(seasep[0:2])
    episode = int(seasep[2:4])
    for items in list:
        begtime = items[0]
        endtime = items[1]
        char = items[2]
        if char:
#             print(char)
            cur.execute('''UPDATE laughs SET char=? WHERE season=? AND episode=? AND beg=? ''',
                (char, season, episode, begtime))
            conn.commit()
    

### Preprocessing for custom subtitles inputs

In [ ]:
customsubdict = {}

for seasep, speaklist in speakdict.items():
#     if seasep == "0101":
    epsubdict = {}
    for speakinfo in speaklist:
        speakbeg = str(int(speakinfo[0]))
        speakend = str(int(speakinfo[1]))
        speakchar = speakinfo[2]
        if speakbeg in epsubdict:
            speakbeg = str(int(speakbeg) + 1)
        epsubdict[speakbeg] = ["speak", speakend, speakchar]
    for laughinfo in laughdict[seasep]:
        laughstart = str(int(laughinfo[0]))
        laughend = str(int(laughinfo[1]))
        laughchar = laughinfo[2]
        if laughstart in epsubdict:
            laughstart = str(int(laughstart) + 1)
        epsubdict[laughstart] = ['laugh', laughend, laughchar]
    customsubdict[seasep] = epsubdict
# print(customsubdict)


### Utility function to convert timestamps into SRT file (subtitle file) format

In [ ]:
def converttosubtime(timestring):
    ms = timestring[-3:]
    rawsecs = timestring[:-3]
    # using int to round down after dividing by 60
    mins = int(int(rawsecs) / 60)
    secs = int(rawsecs) % 60
    hours = int(int(mins) / 60)
    subtime = str(hours).zfill(2) + ':' + str(mins).zfill(2) + ':' + str(secs).zfill(2) + ',' + str(ms).zfill(3)
    return subtime

### Creating custom subtitles file for each episode

SRT files are just text files with a standardized format for displaying subtitles on a video. We convert our timestamps and speaking lines into SRT format and we can feed it right in to any video file. 

In [ ]:
folder = '/Users/Jack/Developer/friends/customsubs/'

for seasep, epsubdict in customsubdict.items():
    beglist = []
    for begtimekey, _ in epsubdict.items():
        begtimeint = int(begtimekey)
        beglist.append(begtimeint)
    beglist.sort()
    file = open(folder + seasep + ".srt", "w")
    count = 1
    for begtime in beglist:
        begtime = str(begtime)
        linetype = epsubdict[begtime][0]
        endtime = epsubdict[begtime][1]
        char = epsubdict[begtime][2]
        if char:
            char = char.capitalize()
        else:
            char = "Undetermined"
        file.write(str(count))
        count += 1
        file.write('\n')
        file.write(converttosubtime(begtime))
        file.write(' --> ')
        file.write(converttosubtime(endtime))
        file.write('\n')
        if linetype == 'speak':
            file.write(char + ' is speaking')
        elif linetype == 'laugh':
            file.write('Laughter (' + char + ' is funny)')
        file.write('\n')
        file.write('\n')
    file.close()

# Getting the actual statistics for the article and graphs

### Getting total number of laughs per character

In [ ]:
cur.execute('''SELECT char FROM laughs''')

laughscount = { 'chandler' : 0,
                'ross' : 0,
                'phoebe' : 0,
                'monica' : 0,
                'rachel' : 0,
                'joey' : 0}

for row in cur:
    dbchar = row[0]
    for char, _ in laughscount.items():
        if dbchar == char:
            laughscount[char] += 1
print(laughscount)

### Getting total amount of laughter generated by each character

In [ ]:
cur.execute('''SELECT char, beg, end FROM laughs''')

laughtime = { 'chandler' : 0.0,
                'ross' : 0.0,
                'phoebe' : 0.0,
                'monica' : 0.0,
                'rachel' : 0.0,
                'joey' : 0.0}

for row in cur:
    dbchar = row[0]
    begtime = float(row[1]) / 1000
    endtime = float(row[2]) / 1000
    for char, _ in laughtime.items():
        if dbchar == char:
            laughtime[char] += (endtime - begtime)
print(laughtime)

### Getting length of laughter on average after each funny character line

In [ ]:
avglaughlen = {}

for char, laughcount in laughscount.items():
    laughlen = laughtime[char]
    lenperlaugh = laughlen / laughcount
    avglaughlen[char] = lenperlaugh
print(avglaughlen)
    

### Getting funniest, least funny episodes by total amount of laughter and total number of laughs

In [ ]:
cur.execute('''SELECT season, episode, beg, end FROM laughs''')
eplaughamount = {}
eplaughnumber = {}

for row in cur:
    season = str(row[0]).zfill(2)
    episode = str(row[1]).zfill(2)
    seasep = season + episode
    begtime = int(row[2]) / 1000
    endtime = int(row[3]) / 1000
    laughlen = endtime - begtime
    
    if seasep not in eplaughnumber:
        eplaughnumber[seasep] = 1
    else:
        eplaughnumber[seasep] += 1
        
    if seasep not in eplaughamount:
        eplaughamount[seasep] = laughlen
    else:
        eplaughamount[seasep] += laughlen
# print(eplaughamount)
# print(eplaughnumber)

fewestlaughs = None
mostlaughs = None
leastlaughter = None
mostlaughter = None

for seasep, laughnumber in eplaughnumber.items():
    
    if not fewestlaughs:
        fewestlaughs = laughnumber
    elif laughnumber < fewestlaughs:
        fewestlaughs = laughnumber
        fewestlaughep = seasep
    if not mostlaughs:
        mostlaughs = laughnumber
    elif laughnumber > mostlaughs and seasep not in ['0212', '0923', '1017']:
        mostlaughs = laughnumber
        mostlaughsep = seasep
        

for seasep, laughamount in eplaughamount.items():       
    if not leastlaughter:
        leastlaughter = laughamount
    elif laughamount < leastlaughter:
        leastlaughter = laughamount
        leastlaughterep = seasep
    if not mostlaughter:
        mostlaughter = laughamount
    elif laughamount > mostlaughter and seasep not in ['0212', '0923', '1017']:
        mostlaughter = laughamount
        mostlaughterep = seasep
        
# print(fewestlaughep, fewestlaughs)
# print(mostlaughsep, mostlaughs)

# print(leastlaughterep, leastlaughter)
# print(mostlaughterep, mostlaughter)
    
    
    


### Getting how funny each season was (by total laughter amount)

In [ ]:
cur.execute('''SELECT season, beg, end FROM laughs''')
seasonlaughamount = {}

for row in cur:
    season = str(row[0]).zfill(2)
    begtime = int(row[1]) / 1000
    endtime = int(row[2]) / 1000
    laughlen = endtime - begtime
    
    if season not in seasonlaughamount:
        seasonlaughamount[season] = laughlen
    else:
        seasonlaughamount[season] += laughlen
# print(seasonlaughamount)

seasepcount = []
for season, numepisodes in seasepdict.items():
    seasepcount.append(numepisodes)
medianepcount = np.median(seasepcount)
# print(medianepcount)

# Normalizing season 10 because it was shorter
# subtracting 1 because season 10 already had its double episode counted as ep 17 (don't want to extrapolate that)
season10multiplier = (medianepcount -1) / seasepdict[10]
oldseason10laughamount = seasonlaughamount['10']
seasonlaughamount['10'] = oldseason10laughamount * season10multiplier
print(seasonlaughamount)

### Plotting funniest and least funny seasons

In [ ]:
seasonlaughamountlist = []
for key, value in seasonlaughamount.items():
    templist = [int(key), int(round(value, 0))]
    seasonlaughamountlist.append(templist)
seasonlaughamountlist.sort(key = lambda x: x[0], reverse=False)
# print(seasonlaughamountlist)
seasonlist = [x[0] for x in seasonlaughamountlist]
laughamountlist = [x[1] for x in seasonlaughamountlist]

# colors = ['yellow'] * 10


seasonlaughrank = go.Figure(data=[go.Bar(
    x=seasonlist,
    y=laughamountlist,
    marker={'color': laughamountlist,
               'colorscale': 'Tealgrn'}
)])


seasonlaughrank.update_xaxes(dtick=1)
seasonlaughrank.update_yaxes(dtick=2000)
seasonlaughrank.update_layout(yaxis_tickformat = 's')
seasonlaughrank.update_layout(title_text='Seconds of Laughter per Season')



### Signing in to plotly and uploading chart

In [ ]:
tls.set_credentials_file(username=username, api_key=api_key)
seasonlaughrankcode = py.plot(seasonlaughrank, filename = 'season_laugh_rank', auto_open=True)
tls.get_embed(seasonlaughrankcode) #change to your url

### Getting longest continuous laughter in the show (funniest moment)

In [ ]:

cur.execute('''SELECT season, episode, char, beg, end FROM laughs''')

longestlaugh = 0
for row in cur:
    season = row[0]
    episode = row[1]
    char = row[2]
    beg = int(row[3])
    end = int(row[4])
    laughlen = end - beg
    if laughlen > longestlaugh:
        longestlaugh = laughlen
        longlaughinfo = [season, episode, char, beg, end]
print(longestlaugh)
print(longlaughinfo)
# Its actually Joey who is funny but the laughter is really 30 secs long and it is Joey chugging a gallon of milk
# because he put it on his resume

### Finding who spoke the most lines in subtitles file (out of the 6 main characters)

In [ ]:
subslinecount = { 'chandler' : 0,
                'ross' : 0,
                'phoebe' : 0,
                'monica' : 0,
                'rachel' : 0,
                'joey' : 0}

cur.execute('''SELECT x, y FROM subs''')

for row in cur:
    char = 'None'
    x = row[0].lower()
    y = row[1].lower()
    if y:
        char = y
    else:
        char = x
    if char in subslinecount:
        subslinecount[char] += 1
print(subslinecount)
        


### Finding who spoke the most lines in script file (out of the 6 main characters)

In [ ]:
scriptlinecount = { 'chandler' : 0,
                'ross' : 0,
                'phoebe' : 0,
                'monica' : 0,
                'rachel' : 0,
                'joey' : 0}

cur.execute('''SELECT char FROM script''')

for row in cur:
    char = row[0].lower()

    if char in scriptlinecount:
        scriptlinecount[char] += 1
print(scriptlinecount)

### Plotting who spoke most lines in the script file

In [ ]:
scriptlinecountlist = []
for key, value in scriptlinecount.items():
    templist = [key.capitalize(), value]
    scriptlinecountlist.append(templist)
scriptlinecountlist.sort(key = lambda x: x[1], reverse=True)
# print(scriptlinecountlist)
charlist = [x[0] for x in scriptlinecountlist]
linecountlist = [x[1] for x in scriptlinecountlist]

colors = [rachelcolor, rosscolor, chandlercolor, monicacolor, joeycolor, phoebecolor]


charlinecount = go.Figure(data=[go.Bar(
    x=charlist,
    y=linecountlist,
    marker_color=colors # marker color can be a single color value or an iterable
)])
charlinecount.update_layout(title_text='Total Lines in Script')




### Signing in to plotly and uploading chart

In [ ]:
tls.set_credentials_file(username=username, api_key=api_key)
charlinecountcode = py.plot(charlinecount, filename = 'char_line_count', auto_open=True)
tls.get_embed(charlinecountcode) #change to your url

### Finding who is funniest (by laughter time) per line spoken

(using script for now as it is 100% accurate instead of 98% accurate)

In [ ]:
funniestperline = {}

for char, linecount in scriptlinecount.items():
    secslaughperline = laughtime[char] / linecount
    funniestperline[char] = secslaughperline
print(funniestperline)

### FUNNIEST CHARACTER (seconds of laughter per line spoken)

In [ ]:
laughsecsperlinelist = []
for key, value in funniestperline.items():
    templist = [key.capitalize(), value]
    laughsecsperlinelist.append(templist)
laughsecsperlinelist.sort(key = lambda x: x[1], reverse=True)
# print(laughsecsperlinelist)
charlist = [x[0] for x in laughsecsperlinelist]
laughsecslist = [x[1] for x in laughsecsperlinelist]

colors = [joeycolor, chandlercolor, phoebecolor, rosscolor, rachelcolor, monicacolor]


laughsecsperline = go.Figure(data=[go.Bar(
    x=charlist,
    y=laughsecslist,
    marker_color=colors # marker color can be a single color value or an iterable
)])
laughsecsperline.update_layout(title_text='Seconds of Laughter per Line')


### Signing in to plotly and uploading chart

In [ ]:
tls.set_credentials_file(username=username, api_key=api_key)
laughsecsperlinecode = py.plot(laughsecsperline, filename = 'laughsecsperline', auto_open=True)
tls.get_embed(laughsecsperlinecode) #change to your url

### Finding who is funniest (by num of laughs) per LINE spoken

(using script for now as it is 100% accurate compared to 98%)

In [ ]:
laughsperline = {}

for char, linecount in scriptlinecount.items():
    secslaughperline = laughscount[char] / linecount
    laughsperline[char] = secslaughperline
print(laughsperline)

### How much non-verbal humor goes unaccounted for?

In [ ]:
cur.execute('''SELECT char FROM laughs''')

nonverbalcount = 0
totalcount = 0
for row in cur:
    totalcount += 1
    if row[0] == None:
        nonverbalcount += 1
print(nonverbalcount / totalcount)


### Finding confidence intervals for our final prediction

In [ ]:
pchar = 0.948 # probability of attributing correct character to laughter instance
plaugh = 0.946 # probability of correctly labeling each timestep of audio
ptotal = pchar * plaugh # probability of getting both correct at each timestep
timesteplen = 11.8 # ms per timestep
z = 1.96 # constant for 95% confidence interval

In [ ]:
# Confidence Intervals for Predicted Values
# https://www.coursera.org/lecture/inferential-statistics/3-08-ci-and-pi-for-predicted-values-77RCt
# How to calculate residual standard deviation
# https://www.investopedia.com/terms/r/residual-standard-deviation.asp

# Reworking formula (1 - 0.9025) squared for all samples doesn't have same magnitude as 1 squared for negative samples
# I think reworking the formula in this way is how the measurements are intended to be
# using avgvalue instead of 1 for the error because if we miss a laugh, we miss by the avgvalue (~ 2 secs), not 1
for char, laughtotal in laughscount.items():
    avglaughvalue = avglaughlen[char]
    part1 = (avglaughvalue) ** 2
    sumofpart1s = part1 * (laughtotal * (1 - ptotal))
#     print(sumofpart1s)
    resSD = np.sqrt(sumofpart1s / (laughtotal - 2))
#     print(resSD)
    CI = z * (resSD / np.sqrt(laughtotal))
#     print(CI)
    print("Confidence interval for avg laughter for " + char + " is plus or minus " + str(CI) + " seconds")
    for funniestlist in laughsecsperlinelist:
        if funniestlist[0].lower() == char:
            funniestlist.append(CI)
print(laughsecsperlinelist)



In [ ]:
# For laughter by season
avgvalue = 1 # when we get a binary value wrong, by definition we are off by 1
errorlist = []

# Reworking formula (1 - 0.95) squared for all samples doesn't have same magnitude as 1 squared for negative samples
# I think reworking the formula in this way is how the measurements are intended to be
for i, seasonlaughter in enumerate(laughamountlist):
    season = i + 1
    laughtertimesteps = seasonlaughter * (1000 / timesteplen) # taking seconds and turning them into 11.8 ms timesteps
    part1 = (avgvalue - 0) ** 2
    sumofpart1s = part1 * (laughtertimesteps * (1 - .95))
#     print(sumofpart1s)
    resSD = np.sqrt(sumofpart1s / (laughtertimesteps - 2))
#     print(resSD)
    CI = z * (resSD / np.sqrt(laughtertimesteps))
#     print(CI)

    # CI for sum is equal to CI for mean * number of timesteps
    # https://stats.stackexchange.com/questions/304999/confidence-intervals-of-the-sum

    sumCI = CI * laughtertimesteps
#     print(sumCI)
    sumCIseconds = sumCI * timesteplen / 1000
    print("CI in seconds for laughter for season " + str(season) + " is " + str(sumCIseconds))
    errorlist.append(sumCIseconds)


### Graphing the Confidence Intervals

In [ ]:
seasonlaughrankerrorbars = go.Figure(data=[go.Bar(
    x=seasonlist,
    y=laughamountlist,
    marker={'color': laughamountlist,
               'colorscale': 'Tealgrn'},
    error_y=dict(type='data', array=errorlist)
)])


seasonlaughrankerrorbars.update_xaxes(dtick=1)
seasonlaughrankerrorbars.update_yaxes(dtick=2000)
seasonlaughrankerrorbars.update_layout(yaxis_tickformat = 's')
seasonlaughrankerrorbars.update_layout(title_text='Seconds of Laughter per Season')


In [ ]:
tls.set_credentials_file(username=username, api_key=api_key)
seasonlaughrankerrorbarscode = py.plot(seasonlaughrankerrorbars, filename = 'seasonlaughrankerrorbars', auto_open=True)
tls.get_embed(seasonlaughrankerrorbarscode) #change to your url

#### Funniest character chart with error bars

In [ ]:
colors = [joeycolor, chandlercolor, phoebecolor, rosscolor, rachelcolor, monicacolor]
errors = [x[2] for x in laughsecsperlinelist]

laughsecsperlineerrorbars = go.Figure(data=[go.Bar(
    x=charlist,
    y=laughsecslist,
    marker_color=colors, # marker color can be a single color value or an iterable
    error_y=dict(type='data', array=errors)
)])
laughsecsperlineerrorbars.update_layout(title_text='Seconds of Laughter per Line')


In [ ]:
tls.set_credentials_file(username=username, api_key=api_key)
laughsecsperlineerrorbarscode = py.plot(laughsecsperlineerrorbars, filename = 'laughsecsperlineerrorbars', auto_open=True)
tls.get_embed(laughsecsperlineerrorbarscode) #change to your url

### Testing Character Laughter by Season

In [ ]:
cur.execute('''SELECT season, char, beg, end FROM laughs''')

laughtimebyseason = { 'chandler' : {},
                'ross' : {},
                'phoebe' : {},
                'monica' : {},
                'rachel' : {},
                'joey' : {}}

for row in cur:
    season = str(row[0])
    dbchar = row[1]
    begtime = float(row[2]) / 1000
    endtime = float(row[3]) / 1000
    for char, _ in laughtimebyseason.items():
        if dbchar == char:
            if season not in laughtimebyseason[char]:
                laughtimebyseason[char][season] = 0
            laughlen = endtime - begtime
            laughtimebyseason[char][season] += laughlen
# print(laughtimebyseason)

cur.execute('''SELECT season, char FROM script''')
    
scriptlinesbyseason = { 'chandler' : {},
                'ross' : {},
                'phoebe' : {},
                'monica' : {},
                'rachel' : {},
                'joey' : {}} 

for row in cur:
    season = str(row[0])
    dbchar = row[1].lower()
    if dbchar in scriptlinesbyseason:
        if season not in scriptlinesbyseason[dbchar]:
            scriptlinesbyseason[dbchar][season] = 0
        scriptlinesbyseason[dbchar][season] += 1
# print(scriptlinesbyseason)

for char, seasondict in laughtimebyseason.items():
    for season, value in seasondict.items():
        laughtimebyseason[char][season] = round(value / scriptlinesbyseason[char][season], 4)
# print(laughtimebyseason)

# Getting laugh time proportions
seasontotals = {}
for char, seasondict in laughtimebyseason.items():
    for season, value in seasondict.items():
        if season not in seasontotals:
            seasontotals[season] = 0
        seasontotals[season] += value
# print(seasontotals)
for char, seasondict in laughtimebyseason.items():
    for season, value in seasondict.items():
        laughtimebyseason[char][season] = round(value / seasontotals[season], 4)
# print(laughtimebyseason)

# Convert to DF
dictforDF = {"Season" : [], 
             "Friend" : [], 
             "% Share of Laughter per Line" : []}
for char, seasondict in laughtimebyseason.items():
    for season, value in seasondict.items():
        dictforDF["Season"].append(int(season))
        dictforDF["Friend"].append(char.capitalize())
        dictforDF["% Share of Laughter per Line"].append(value)
# print(dictforDF)
df = pd.DataFrame(dictforDF)
dfsorted = df.sort_values(by=['Season'])
# print(dfsorted)

# Build and show graph
laughterperlineovertime = px.line(dfsorted, x='Season', y='% Share of Laughter per Line', color='Friend', 
              color_discrete_map={"Joey":joeycolor,
                                  "Chandler":chandlercolor,
                                  "Monica":monicacolor,
                                  "Rachel":rachelcolor,
                                  "Phoebe":phoebecolor,
                                  "Ross":rosscolor,
                                 })
laughterperlineovertime.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'title_text':'Laughter Generated By Season',
'legend_title_text' : None,
'legend_orientation' : 'v',
'yaxis': {
    'tickformat': ',.0%'
  }
})
laughterperlineovertime.update_xaxes(dtick=1)
laughterperlineovertime.show()

    

In [ ]:
tls.set_credentials_file(username=username, api_key=api_key)
laughterperlineovertimecode = py.plot(laughterperlineovertime, filename = 'laughterperlineovertime', auto_open=True)
tls.get_embed(laughterperlineovertimecode) #change to your url